In [137]:
import json
import os
import sys

In [138]:
#load json file
def load_json(file):
    with open(file, 'r') as f:
        return json.load(f)

In [139]:
mattrife_posts_ig = load_json("all_instagram_data.json")

In [140]:
mattrife_posts_ig.keys()

mattrife_keylist = list(mattrife_posts_ig.keys())

In [142]:
cleaned_posts = {
    k: v for k, v in mattrife_posts_ig.items()
    if isinstance(v, dict) and 'items' in v and isinstance(v['items'], list) and len(v['items']) > 0
}
first_post_data = [v['items'][0] for v in cleaned_posts.values()]
with open("cleaned.json", "w") as f:
    json.dump(cleaned_posts, f, indent=2)


⚠️ No 'items' found for cristiano
Raw content: {'/cristiano/p/DIe1T1bg2Wf/': {'num_results': 1, 'more_available': False, 'items': [{'pk': '3611558410702775711', 'id': '3611558410702775711_173560420', 'fbid': '18110317018438334', 'device_timestamp': 1744751230842470, 'caption_is_edited': False, 'strong_id__': '3611558410702775711_173560420', 'deleted_reason': 0, 'has_shared_to_fb': 3, 'has_delayed_metadata': False, 'mezql_token': '', 'share_count_disabled': False, 'should_request_ads': False, 'is_reshare_of_text_post_app_media_in_ig': False, 'integrity_review_decision': 'pending', 'client_cache_key': 'MzYxMTU1ODQxMDcwMjc3NTcxMQ==.3', 'is_visual_reply_commenter_notice_enabled': True, 'like_and_view_counts_disabled': False, 'is_post_live_clips_media': False, 'can_modify_carousel': True, 'is_quiet_post': False, 'has_privately_liked': False, 'filter_type': 0, 'subtype_name_for_REST__': 'XDTCarouselContainerMedia', 'taken_at': 1744751334, 'usertags': {'in': [{'position': [0.6179487179, 0.576

In [121]:
post0_json = mattrife_posts_ig[mattrife_keylist[POST_NUM]]['items'][0]

KeyError: 'items'

#### Desired data pairs and metadata

In [122]:
#declare a dictionary to store the post info
post0 = {}

In [123]:
post0['id'] = post0_json['id']
post0['shortcode'] = post0_json['code'] #shortcode id
post0['caption'] = post0_json['caption']['text'] #caption

TypeError: 'NoneType' object is not subscriptable

In [124]:
#We're interested in only the first (display) image
post0['head_image_url'] = post0_json['image_versions2']['candidates'][0]['url']

In [125]:
#Also interested in music metadata
music_metadata = post0_json['music_metadata']
post0_json['music_metadata']

{'audio_type': 'licensed_music',
 'music_canonical_id': '18147793093020237',
 'pinned_media_ids': [],
 'music_info': {'music_canonical_id': None,
  'music_asset_info': {'allows_saving': True,
   'artist_id': '341366006653245',
   'audio_asset_id': '186510556490141',
   'audio_cluster_id': '2417453701611662',
   'cover_artwork_thumbnail_uri': 'https://scontent-lax3-1.xx.fbcdn.net/v/t39.30808-6/427578120_2601197316721874_4005172846330706337_n.jpg?stp=dst-jpg_s168x128_tt6&_nc_cat=1&ccb=1-7&_nc_sid=2f2557&_nc_ohc=9DjkHuwHLjYQ7kNvwFgLzW7&_nc_oc=AdnI_pgVFTlaGy0aCdeViSarFOgSD5gfnyFKpTKGSVGGmBASHUf4vb-LgKTx2CIvvgw&_nc_ad=z-m&_nc_cid=0&_nc_zt=23&_nc_ht=scontent-lax3-1.xx&_nc_gid=1xsTpb7FgmU03K9iDvDV6w&oh=00_AfHFnB8pQwq9mlN082EzYSPyk56Yi3ChW89WomWyxcPkqg&oe=6804DF3B',
   'cover_artwork_uri': 'https://scontent-lax3-1.xx.fbcdn.net/v/t39.30808-6/427578120_2601197316721874_4005172846330706337_n.jpg?stp=dst-jpg_s261x260_tt6&_nc_cat=1&ccb=1-7&_nc_sid=2f2557&_nc_ohc=9DjkHuwHLjYQ7kNvwFgLzW7&_nc_oc=AdnI_

In [126]:
post0['audio_type'] = music_metadata['audio_type']
music_metadata['audio_type'] == 'licensed_music'
#We want that to be true


True

In [127]:
music_info = music_metadata['music_info']

In [128]:
music_info['music_asset_info']

{'allows_saving': True,
 'artist_id': '341366006653245',
 'audio_asset_id': '186510556490141',
 'audio_cluster_id': '2417453701611662',
 'cover_artwork_thumbnail_uri': 'https://scontent-lax3-1.xx.fbcdn.net/v/t39.30808-6/427578120_2601197316721874_4005172846330706337_n.jpg?stp=dst-jpg_s168x128_tt6&_nc_cat=1&ccb=1-7&_nc_sid=2f2557&_nc_ohc=9DjkHuwHLjYQ7kNvwFgLzW7&_nc_oc=AdnI_pgVFTlaGy0aCdeViSarFOgSD5gfnyFKpTKGSVGGmBASHUf4vb-LgKTx2CIvvgw&_nc_ad=z-m&_nc_cid=0&_nc_zt=23&_nc_ht=scontent-lax3-1.xx&_nc_gid=1xsTpb7FgmU03K9iDvDV6w&oh=00_AfHFnB8pQwq9mlN082EzYSPyk56Yi3ChW89WomWyxcPkqg&oe=6804DF3B',
 'cover_artwork_uri': 'https://scontent-lax3-1.xx.fbcdn.net/v/t39.30808-6/427578120_2601197316721874_4005172846330706337_n.jpg?stp=dst-jpg_s261x260_tt6&_nc_cat=1&ccb=1-7&_nc_sid=2f2557&_nc_ohc=9DjkHuwHLjYQ7kNvwFgLzW7&_nc_oc=AdnI_pgVFTlaGy0aCdeViSarFOgSD5gfnyFKpTKGSVGGmBASHUf4vb-LgKTx2CIvvgw&_nc_ad=z-m&_nc_cid=0&_nc_zt=23&_nc_ht=scontent-lax3-1.xx&_nc_gid=1xsTpb7FgmU03K9iDvDV6w&oh=00_AfFXRqoCnlBYHBmyoXzjL

In [129]:
post0['music_title'] = music_info['music_asset_info']['title']
post0['music_artist'] = music_info['music_asset_info']['display_artist']

#### Parsing and Projecting with JMESPath

In [79]:
!python3 -m pip install jmespath



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip


In [130]:
import jmespath

def parse_post_json(data: dict) -> dict:
    """Parse json data from instagram post and return a dictionary of relevant information"""
    result = jmespath.search(
        """{
        id: id,
        shortcode: code,
        caption: caption.text,
        head_image_url: image_versions2.candidates[0].url,
        audio_type: music_metadata.audio_type,
        music_title: music_metadata.music_info.music_asset_info.title,
        music_artist: music_metadata.music_info.music_asset_info.display_artist
    }""",
        data,
    )
    return result

In [131]:
parse_post_json(post0_json)

{'id': '3607792429048554130_654228559',
 'shortcode': 'DIRdBlsyGKS',
 'caption': None,
 'head_image_url': 'https://scontent-lax3-1.cdninstagram.com/v/t51.2885-15/489427087_18493689643020560_3177847082020357242_n.jpg?stp=dst-jpg_e35_p1080x1080_tt6&cb=30a688f7-fa102a98&efg=eyJ2ZW5jb2RlX3RhZyI6IkNBUk9VU0VMX0lURU0uaW1hZ2VfdXJsZ2VuLjE0NDB4MTgwMC5zZHIuZjc1NzYxLmRlZmF1bHRfaW1hZ2UifQ&_nc_ht=scontent-lax3-1.cdninstagram.com&_nc_cat=108&_nc_oc=Q6cZ2QGReXnxckOkI5EEwExjNKyE1ukbfqrkddbGD6GaLWDgjDc3RWmM4XBlL5Dfom_JGao&_nc_ohc=nEbG7-uG1TIQ7kNvwG0YgPR&_nc_gid=1xsTpb7FgmU03K9iDvDV6w&edm=ALQROFkBAAAA&ccb=7-5&ig_cache_key=MzYwNzc5MjQyMTc1MDU0MzEyOA%3D%3D.3-ccb7-5-cb30a688f7-fa102a98&oh=00_AfHF_4-gTj_gwUdWBiyHJr5qj8SnVM4XKd5mLoqMbePNSA&oe=6804E52A&_nc_sid=fc8dfb',
 'audio_type': 'licensed_music',
 'music_title': 'needy',
 'music_artist': 'Ariana Grande'}

In [132]:
# for all posts, parse them, and save them to a new json file
all_posts = []
for post in mattrife_posts_ig.values():
    all_posts.append(parse_post_json(post['items'][0]))

with open('mattrife_posts_parsed.json', 'w') as f:
    json.dump(all_posts, f, indent=4)


KeyError: 'items'

In [83]:
!python3 -m pip install pandas



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip


In [133]:
# alternatively, for all posts, save into a pandas DF, and then into a csv
import pandas as pd

df = pd.DataFrame(all_posts)
df.to_csv('all_instagram_data.csv', index=False)

In [134]:
df

""


In [135]:
df.columns

RangeIndex(start=0, stop=0, step=1)

In [112]:
# for all "None" on audio type, remove them
df = df[df['audio_type'] == 'licensed_music']

In [113]:
# remove id, caption, audio type, 
df = df.drop(columns=['id', 'caption', 'audio_type'])

In [114]:
df

,shortcode,head_image_url,music_title,music_artist
0,DIRdBlsyGKS,https://scontent-lax3-1.cdninstagram.com/v/t51...,needy,Ariana Grande
1,DB6vRMEy9cz,https://scontent-lax3-1.cdninstagram.com/v/t51...,Bed Chem,Sabrina Carpenter
5,C7zpabtyVOy,https://scontent-lax3-1.cdninstagram.com/v/t51...,How Sweet,NewJeans


In [115]:
#save df to csv
df.to_csv('all_instagram_data.csv', index=False)